# **Advanced Lane Finding Project**
The goals / steps of this project are the following:
* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)
[image0]: ./final_results/dist_corr.jpeg 
[image1]: ./final_results/dist_corr2.jpeg
[image2]: ./final_results/binary.jpeg 
[image3]: ./final_results/Warped.jpeg
[image4]: ./final_results/poly.jpeg 
[image5]: ./final_results/final.jpeg 
[image6]: ./final_results/verify.jpeg 



## **Rubric Points**

### **Camera Calibration**

**Have the camera matrix and distortion coefficients been computed correctly and checked on one of the calibration images as a test?**

Calibration was done using the chessboard images provided in the repository. So firstly the object points and image points are calculated. The object points are the (x,y,z) (here assuming z to be 0)coordinates of the chessboard corners. The chessboard corners will form a grid of 9x6, so a array is made with the coordinates of the corners forming a grid. The image points are the actual pixel positions of these corners and is calculated with the help of cv2.findChessboardCorners(). These object points and image points are then used to calculate the camera calibration matrix using the cv2.calibrateCamera() function. I applied this distortion correction to the test image using the cv2.undistort() function and obtained this result:

![Camera calibration output][image0]

### **Pipeline(single images)**

**1. Has the distortion correction been correctly applied to each image?**
Here is the output of distortion correction along with the original image:

![Distortion Correction][image1]

**2. Has a binary image been created using color transforms, gradients or other methods?**
Here is a binary image of one of the test images generated.
In the final binary image a value 1 is given if for that pixel gradx==1 and dir_binary==1 and s_binary==1 where grad_x is gradient thresholded image along x axis, dir_binary is the thresholded image w.r.t the gradient direction and s_binary is the thresholded image w.r.t the saturation channel of the HLS converted image(Used for yellow detection, Gradient enough to detect white).

![Binary Image][image2]

**3. Has a perspective transform been applied to rectify the image?**
Yes, The functions get_transform_matrix() gets the transformation matrix by using the cv2.getPerspectiveTransform() function. The source and destination points provided are as follows: 

|Source    | Destination   |
|-         |-              |
|(275,673) |(290,720)      |
|          |               |
|(589,455) |(290,0)        |
|          |               |
|(694,455) |(990,0)        |
|          |               |
|(1032,673)|(990,720)      |

The function image_warp uses this matrix to warp images. The output is as follows:

![Warped image][image3]

I verified that my perspective transform was working as expected by drawing the src and dst points onto
a test image and its warped counterpart to verify that the lines appear parallel in the warped image

![Verification][image6]

**4. Have lane line pixels been identified in the rectified image and fit with a polynomial?**
The sliding window approach is used to detect the lanes. Firstly we create a histogram and detect the two peaks and identify those 2 peaks(one left lane and one right) as our starting points. A box is drawn around each peak and is and inside this box all the nonzero points(the image is binary) are stored in a array called good_left_inds. A minimum threshold is set for the number of points found in the rectangle to change the lane center. If the number of nonzero points in the rectangle exceeds the threshold, we set the centroid of these points as the new center for the rectangle. The width of this rectangle is equal to 2 * margin. This way we slide up the image and all lane points are stored in leftx and lefty and rightx and righty. This array is then used as input to np.polyfit to get the two lanes as polynomial functions. We are given the values ym_per_pix and xm_per_pix as well which are used during the calculation of radius of curvature and distance from center. These values help convert pixel values to actual real world distances. The output is as follows:

![Warped lanes detected][image4]

**5. Having identified the lane lines, has the radius of curvature of the road been estimated? And the position of the vehicle with respect to center in the lane?**
Yes we find the polynomials by myltiplying leftx,lefty,rightxand righty ny ym_per_pix and xm_per_pix to convert them to real distances and then using the formulas in the course the distance and radius of curvature are calculated and displayed on the image:

![Final_output][image5]

### **Pipeline(video)**
**1. Does the pipeline established with the test images work to process the video?**
Yes it works perfectly on the project_video.mp4. The result is present as project_video_output.mp4. It makes many mistakes on the other videos.

### **Discussion**
My approach is very clearly listed in the previous questions, so I'll give a outline of the pipeline:

* First camera calibration is done via the chessboard images provided. The camera matrix obtained via doing this is used to undistort the future images.
* The undistorted image goes through 3 computations to generate the binary image:
    * First, the gradient along the x axis is thresholded.
    * Second, Gradient direction is thresholded.
    * Third the image is converted to HLS and the S-channel is used for thresholding.
* The values for these thresholds are obtained experimentally(whatever worked best!). These are then combined via the formula shown above.
* This image is then warped and then on the warped image, we use the histogram to find starting points and then we use the sliding window approach to find lanes in the rest of the image. **A low pass filter is used in the video so that outliers don't lead to sharp changes in the lanes detected.**
* The radius of curvature and distance from center are calculated.
* The lanes are warped via the inverse matrix and then overlayed on the 'image.

There were many challenges:
* In the challenge videos, The divider poses great challenges, as it is always caught due to the sharp gradient there, leading to a poor results.
* The colour thresholding cannot filter out shadows that well. leading to some bad results in shadows.
* The pipeline is computationally expensive and takes a lot of time to detect the lanes.

Improvements:
* Greater integration of colour in the detection of lanes(I am reffering to the combination function of gradient and colour) will lead to much better results on challenge and harder video.
* Greater shadow filtration can be done. I found many research papers tackling this issue. 